# Coleta de endereços

Essa coleta de endereços serve para enriquecer os dados da análise
da violência em Feira de Santana. Os dados foram conseguidos com a 
Secretaria de Segurança Pública da Bahia, através da Lei de Acesso
à Informação.

Aqui utilizamos a [HERE API](https://developer.here.com/develop/rest-apis)
para coletar os endereços (em seu plano gratuito). Para rodar esse código
basta ter o CSV com os dados de boletins de ocorrência (que poderão ser
encontrados em nosso [Kaggle](https://www.kaggle.com/datasets/dadosabertosdefeira/boletins-de-ocorrncia-em-feira-20172020) e ter
a chave da API na variável de ambiente `HERE_API_KEY`. Para conseguir
a chave basta fazer um cadastro no site da HERE.

In [ ]:
import pandas as pd
import json


df = pd.read_csv("2020-FEIRA - bo_feira_20172019.csv")
df.head()

In [ ]:
addresses = df[['dbo_Logradouro_descricao', 'dbo_Bairro_descricao']]
addresses

In [ ]:
addresses = addresses[addresses['dbo_Logradouro_descricao'] != 'NAO INFORMADO']
addresses

In [ ]:
unique_address = {}
for _, row in addresses.iterrows():
    if not unique_address.get(row['dbo_Logradouro_descricao']):
        unique_address[row['dbo_Logradouro_descricao']] = {row['dbo_Bairro_descricao']: True}
    elif not unique_address[row['dbo_Logradouro_descricao']].get(row['dbo_Bairro_descricao']):
        unique_address[row['dbo_Logradouro_descricao']] = {row['dbo_Bairro_descricao']: True}
    else:
        unique_address[row['dbo_Logradouro_descricao']][row['dbo_Bairro_descricao']] = True

print(len(unique_address))
with open("unique_addresses.txt", 'w') as fout:
    json_dumps_str = json.dumps(unique_address, indent=4)
    print(json_dumps_str, file=fout)
unique_address

In [ ]:
from scripts.geolocation_info import get_geocode_info
import time


start = 0
count = 0

for street, neighborhoods in unique_address.items():
    if start <= count:
        for neighborhood in neighborhoods.keys():
            full_address = f"{street}, {neighborhood}, Feira de Santana, Bahia, Brazil"
            try:
                payload = get_geocode_info(full_address, raise_exception=True)
            except:
                print("Time to sleep...")
                time.sleep(30)
                payload = get_geocode_info(full_address, raise_exception=True)
            if payload:
                for result in payload:
                    if result['address']['city'] == "Feira de Santana":
                        unique_address[street][neighborhood] = {
                            'api_address': result['title'],
                            'api_lat': result['position']['lat'],
                            'api_long': result['position']['lng'],
                        }
                        print(f"{count} {full_address}")
                        break
    count += 1


In [ ]:
def get_address(row):
    try:
        return unique_address[row.dbo_Logradouro_descricao][row.dbo_Bairro_descricao]['api_address']
    except:
        pass
    return

def get_lat(row):
    try:
        return unique_address[row.dbo_Logradouro_descricao][row.dbo_Bairro_descricao]['api_lat']
    except:
        pass
    return

def get_lng(row):
    try:
        return unique_address[row.dbo_Logradouro_descricao][row.dbo_Bairro_descricao]['api_long']
    except:
        pass
    return


addresses['api_address'] = addresses.apply(get_address, axis=1)
addresses['api_lat'] = addresses.apply(get_lat, axis=1)
addresses['api_lng'] = addresses.apply(get_lng, axis=1)

In [ ]:
addresses

In [ ]:
addresses.to_csv('api-unique-addresses-feira-2017-2019.csv')